In [1]:
import geopandas as gpd
import pandas as pd
from tqdm.std import tqdm

In [2]:
USA_EQUAL_AREA = 'PROJCS["US_National_Atlas_Equal_Area",GEOGCS["GCS_Sphere_Clarke_1866_Authalic",DATUM["D_Sphere_Clarke_1866_Authalic",SPHEROID["Sphere_Clarke_1866_Authalic",6370997.0,0.0]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Azimuthal_Equal_Area"],PARAMETER["latitude_of_center",45.0],PARAMETER["longitude_of_center",-100.0],PARAMETER["false_easting",0.0],PARAMETER["false_northing",0.0],UNIT["Meter",1.0]]'

In [ ]:
states = gpd.read_parquet(
    "data/us-state-boundaries.parquet", columns=["name", "geo_point_2d", "st_asgeojson"]
)

original_crs = states.crs
states.drop(columns=["geo_point_2d"], inplace=True)
states.rename(columns={"st_asgeojson": "geometry"}, inplace=True)
states.set_geometry("geometry", inplace=True)
states.set_crs(original_crs, inplace=True)

states["name"] = states["name"].str.lower().str.replace(" ", "_")

states["geometry"] = states.to_crs(USA_EQUAL_AREA).simplify(9).to_crs("EPSG:4326")

states.sample(10, random_state=42)

In [4]:
not_valid_states = [
    "puerto_rico",
    "commonwealth_of_the_northern_mariana_islands",
    "american_samoa",
    "district_of_columbia",
    "guam",
    "united_states_virgin_islands",
]

states = states[~states.name.isin(not_valid_states)].reset_index(drop=True)


In [ ]:
states = states[states.name == "california"].reset_index(drop=True)
states

In [ ]:
for idx, state in states.iterrows():
    state_name = state['name']
    state_geom = state.geometry

    state_gdf = gpd.GeoDataFrame([state], columns=states.columns, crs=states.crs)[["geometry"]]

    plots = gpd.read_parquet("data/varda-fieldid-2024-07-USA_0.parquet")
    final_gdf = gpd.overlay(plots, state_gdf, how='intersection')

    for i in tqdm(range(1, 29)):
        plots = gpd.read_parquet(f"data/varda-fieldid-2024-07-USA_{i}.parquet")
        intersected_gdf = gpd.overlay(plots, state_gdf, how='intersection')
        final_gdf = pd.concat([final_gdf, intersected_gdf], ignore_index=True)

    final_gdf = final_gdf.sort_values(by="ha", ascending=False).reset_index(drop=True)
    final_gdf = gpd.GeoDataFrame(final_gdf, crs=final_gdf.crs)
    final_gdf.to_parquet(f"data/usa_delineation/{state_name}.parquet", compression="brotli")

    print(state_name)
    break